**Created on**: 2021.09.04

**Implemented by**:Anthony Cho

**Subject**: Quadratic programming: Quadratic Knapsack

### Problem

Given a set of $N$ elements. Each element has its own weight $w_{i}$, a profit $b_{i}$, and an extra profit $p_{ij}$ if the element $i$ and element $j$ are selected. The goal is to decide which element to take into the knapsack in order to maximize the total profit.

### Model:

$$\max \sum_{i=1}^{N} b_i x_i + \sum_{i=1}^{N} \left[ \sum_{j=1, j\neq i}^{N} p_{ij}x_i x_j \right] $$
s.a. $$\sum_{i=1}^{N} w_i x_i \leq W,$$ $$\textbf{x} \in \{0,1\}.$$

In [1]:
## Libraries dependencies

import gurobipy as gp
from numpy import array

### Data

In [2]:
## Knapsack's maximum weight
W = 50

## Number of elements available
N = 15

## Profit of each element
b = array([7, 6, 13,16, 5, 10, 9, 23, 18, 12, 9, 22, 17, 32, 8])

## Weight of each element
w = array([13, 14, 14, 15, 15, 9, 26, 24, 13, 11, 9, 12, 25, 12, 26])

## Extra profit of pair of elements (matrix)
p = array([7, 12, 7, 6, 13, 8, 11 , 7 , 15 , 23, 14, 15, 17, 9, 15,
              12, 6, 15, 13, 10, 15, 9, 10, 8, 17, 11, 13, 12, 16, 15,
              7, 15, 13, 11, 16, 6, 8, 14, 13, 4, 14, 8, 15, 9, 16,
              6, 13, 11, 16, 10, 13, 14, 14, 17, 15, 14, 6, 24, 13, 4,
              13, 10, 16, 10, 5, 9, 7, 25, 12 , 6, 6, 16, 10, 15, 14,
              8, 15, 6, 13, 9, 10, 2, 13, 12, 16, 9, 11, 23, 10, 21,
              11, 9, 8, 14, 7, 2, 9, 8, 18, 4, 13, 14, 14, 17, 15,
              7, 10, 14, 14, 25, 13, 8, 23, 9, 16, 12, 3, 14, 14, 27,
              15, 8, 13, 17, 12, 12, 18, 9, 18, 15, 16, 13, 14, 7, 17,
              23, 17, 4, 15, 6, 16, 4, 16, 15, 12, 28, 5, 19, 6, 18,
              14, 11, 14, 14, 6, 9, 13, 12, 16, 28, 9, 13, 4, 13, 16,
              15, 13, 8, 6, 16, 11, 14, 3, 13, 5, 13, 22, 11, 19, 13,
              17, 12, 15, 24, 10, 23, 14, 14, 14, 19, 4, 11, 17, 15, 12,
              9, 16, 9, 13, 15, 10, 17, 14, 7, 6, 13, 19, 15, 32, 16,
              15, 15, 16, 4, 14, 21, 15, 27, 17, 18, 16, 13, 12, 16, 8
    ]).reshape(N, N)

In [3]:
p

array([[ 7, 12,  7,  6, 13,  8, 11,  7, 15, 23, 14, 15, 17,  9, 15],
       [12,  6, 15, 13, 10, 15,  9, 10,  8, 17, 11, 13, 12, 16, 15],
       [ 7, 15, 13, 11, 16,  6,  8, 14, 13,  4, 14,  8, 15,  9, 16],
       [ 6, 13, 11, 16, 10, 13, 14, 14, 17, 15, 14,  6, 24, 13,  4],
       [13, 10, 16, 10,  5,  9,  7, 25, 12,  6,  6, 16, 10, 15, 14],
       [ 8, 15,  6, 13,  9, 10,  2, 13, 12, 16,  9, 11, 23, 10, 21],
       [11,  9,  8, 14,  7,  2,  9,  8, 18,  4, 13, 14, 14, 17, 15],
       [ 7, 10, 14, 14, 25, 13,  8, 23,  9, 16, 12,  3, 14, 14, 27],
       [15,  8, 13, 17, 12, 12, 18,  9, 18, 15, 16, 13, 14,  7, 17],
       [23, 17,  4, 15,  6, 16,  4, 16, 15, 12, 28,  5, 19,  6, 18],
       [14, 11, 14, 14,  6,  9, 13, 12, 16, 28,  9, 13,  4, 13, 16],
       [15, 13,  8,  6, 16, 11, 14,  3, 13,  5, 13, 22, 11, 19, 13],
       [17, 12, 15, 24, 10, 23, 14, 14, 14, 19,  4, 11, 17, 15, 12],
       [ 9, 16,  9, 13, 15, 10, 17, 14,  7,  6, 13, 19, 15, 32, 16],
       [15, 15, 16,  4, 14, 21, 15

### Model

In [4]:
## Model instance
model = gp.Model('QP Knapsack')

## Creating decision variables: Select or not the element i, for all i
variables = []
for i in range(N):
    variables.append( model.addVar(lb=0, vtype=gp.GRB.BINARY, name='item[{}]'.format(i)) )
variables = array(variables)

## Update the model
model.update()

Academic license - for non-commercial use only - expires 2021-10-22
Using license file /home/hp/gurobi.lic


In [5]:
## Objective function building
fObj = b.dot(variables) + variables.dot(p).dot(variables)

## Objective function and model sense assignment
model.setObjective(fObj, gp.GRB.MAXIMIZE)

## Update the model
model.update()

In [6]:
## Constraint: Knapsack weight limit
constrPeso = model.addConstr(w.dot(variables), gp.GRB.LESS_EQUAL, W, name='LimitePeso')

## Update the model
model.update()

In [7]:
## Model display
model.display()

Maximize
   <gurobi.QuadExpr: 7.0 item[0] + 6.0 item[1] + 13.0 item[2] + 16.0 item[3] + 5.0 item[4] + 10.0 item[5] + 9.0 item[6] + 23.0 item[7] + 18.0 item[8] + 12.0 item[9] + 9.0 item[10] + 22.0 item[11] + 17.0 item[12] + 32.0 item[13] + 8.0 item[14] + [ 7.0 item[0] ^ 2 + 24.0 item[0] * item[1] + 14.0 item[0] * item[2] + 12.0 item[0] * item[3] + 26.0 item[0] * item[4] + 16.0 item[0] * item[5] + 22.0 item[0] * item[6] + 14.0 item[0] * item[7] + 30.0 item[0] * item[8] + 46.0 item[0] * item[9] + 28.0 item[0] * item[10] + 30.0 item[0] * item[11] + 34.0 item[0] * item[12] + 18.0 item[0] * item[13] + 30.0 item[0] * item[14] + 6.0 item[1] ^ 2 + 30.0 item[1] * item[2] + 26.0 item[1] * item[3] + 20.0 item[1] * item[4] + 30.0 item[1] * item[5] + 18.0 item[1] * item[6] + 20.0 item[1] * item[7] + 16.0 item[1] * item[8] + 34.0 item[1] * item[9] + 22.0 item[1] * item[10] + 26.0 item[1] * item[11] + 24.0 item[1] * item[12] + 32.0 item[1] * item[13] + 30.0 item[1] * item[14] + 13.0 item[2] ^ 2 + 22.0

In [8]:
## Optimize the model
model.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (linux64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 1 rows, 15 columns and 15 nonzeros
Model fingerprint: 0x281937d7
Model has 120 quadratic objective terms
Variable types: 0 continuous, 15 integer (15 binary)
Coefficient statistics:
  Matrix range     [9e+00, 3e+01]
  Objective range  [5e+00, 3e+01]
  QObjective range [8e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+01, 5e+01]
Found heuristic solution: objective -0.0000000
Presolve time: 0.01s
Presolved: 106 rows, 120 columns, 330 nonzeros
Variable types: 0 continuous, 120 integer (120 binary)

Root relaxation: objective -6.466981e+02, 89 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  646.69811    0   20   -0.00000  646.69811      -     -    0s
H    0     0             

In [9]:
## Optimal objective value
print('Total profit: {}'.format(model.ObjVal))

## Slack
print('Knapsack free weight remaining: {}'.format(constrPeso.Slack))

## Reporte de elementos considerados
print('Elements to take into the knapsack are: ', end='')
for i, item in enumerate(variables):
    if item.x > 0:
        print(i, end=' ')
    

Total profit: 324.0
Knapsack free weight remaining: 4.0
Elements to take into the knapsack are: 8 10 11 13 